## Conda environment
- conda create -n [name] python=[python_version]
- conda activate [name]

## Local File Connection

### Initialization of Spark

In [ ]:
!pip install jupyterlab
!pip install pyspark --no-cache-dir

In [ ]:
import os
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-1.8"
os.environ['PYSPARK_PYTHON'] = 'python'

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext

In [ ]:
conf = SparkConf() \
    .setAppName("Spark Sample") \
    .setMaster("local") \
    .set("spark.driver.extraClassPath","C:/pyspark/*")

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

### Read the local csv

In [ ]:
df=spark.read.options(delimiter=",", header=True).csv("./sales_generated.csv")

In [ ]:
df.show(5, truncate=False)

In [ ]:
df.createOrReplaceTempView("national_sales")

In [ ]:
output = spark.sql("SELECT * FROM national_sales WHERE market_area = 'Jawa Tengah' LIMIT 10")
output.show()

In [ ]:
"""Write data into Postgres"""
dest_tbl = 'public."pyspark_sales_data"'
database = "postgres"
password = "password"
user = "postgres"

In [ ]:
output.write.mode("overwrite") \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://localhost:5432/{database}") \
    .option("dbtable", dest_tbl) \
    .option("user", user) \
    .option("password", password) \
    .option("driver",  "org.postgresql.Driver") \
    .save()

In [ ]:
spark.stop()

## Local Hadoop Connections

In [ ]:
import os

os.environ['PYSPARK_PYTHON'] = 'python'
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-1.8"

In [ ]:
from pyspark.sql import SparkSession

# Specify the Hadoop user in the SparkSession configuration
spark = SparkSession.builder \
    .appName("Write ORC to HDFS") \
    .config("spark.yarn.submit.file.replication", "1") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .getOrCreate()

In [ ]:
# Example DataFrame
data = [("John", 28), ("Doe", 23), ("Alice", 34)]
columns = ["Name", "Age"]
df = spark.createDataFrame(data, columns)


In [ ]:
# Write ORC file to HDFS
df.write.orc("hdfs://localhost:9000/orc_files", mode="overwrite")

In [ ]:
spark.stop()